In [74]:
import random
import math
import numpy as np
from scipy.optimize import fmin
from training_data import generate_train_data

## trainingsdaten generieren

In [75]:
training_data = generate_train_data(10, 3) # 10 pixelbilder generieren welche 3X3 pixel gross sind

training_data_sample = training_data[0]
training_data_sample_target = [1, 0] if training_data[0][1] == 1 else [0, 1]
training_data_sample_image = training_data[0][0]

print(training_data_sample) # beispiel einer horizontale

([1, 0, 0, 1, 0, 0, 1, 0, 0], 0)


## Netzwerk initialisieren
- funktionen definieren
- funktionen aufrufen

In [76]:
# funktion für fake weights und fake biases eines neurons
def init_bias():
    bias = random.uniform(-0.5, 0.5)
    return bias
#print(init_bias())

def init_weights(anz_weights):
    weights = [random.uniform(-0.5, 0.5) for _ in range(anz_weights)]
    return weights
#print(init_weights(4))


def init_network(dimension):

    network = [] # eine liste von listen von dictionaries (jedes neuron wird von einem dicrionary repräsentiert)
    
    # ----- Input layer ----- (hat keine biases oder weights!)
    # für jedes layer eine liste machen und diese mit den dictionaries füllen
    for layer in dimension[:1]: # nur für die input layers
        network.append([
            {
                "weights":None, 
                "bias":None, 
                "activation":None
            } for _ in range(layer)
        ]) # liste mit leeren dictionaries hinzufügen für jedes neuron des inputlayers

    # ----- Hidden layers -----
    # Für jedes Hidden-Layer eine Liste mit Dictionaries hinzufügen, diese haben weights und biases
    for index, layer in enumerate(dimension[1:-1], start=1):  # i startet bei 1, weil wir ab der 2. Schicht zählen
        network.append([
        {
            "weights": init_weights(dimension[index - 1]),  # Anzahl Gewichte = Anz. Neuronen im vorherigen Layer
            "bias": init_bias(),
            "activation": None
        } for _ in range(layer)  # Anzahl Neuronen in der aktuellen Schicht
    ])
        
    # ----- Output layer -----
    # Für das otput Layer eine Liste mit Dictionaries hinzufügen, diese haben weights und biases
    for index, layer in enumerate(dimension[-1:], start=-1):  # i startet bei 1, weil wir ab der 2. Schicht zählen
        network.append([
        {
            "weights": init_weights(dimension[index - 1]),  # Anzahl Gewichte = Anz. Neuronen im vorherigen Layer
            "bias": init_bias(),
            "activation": None
        } for _ in range(layer)  # Anzahl Neuronen in der aktuellen Schicht
    ])
    return network
#print(init_network([9, 5, 5, 2]))

In [77]:
network_dimension = [9, 5, 5, 2]

network1 = init_network(network_dimension) # netzerkt nach dem n-1 ten durchlauf

# funktion um das netzwerk schöner darzustellen
def print_array_structure(array):
    for i, layer in enumerate(array):
        print(f"🔹 Ebene {i}:")
        for j, element in enumerate(layer):
            print(f"  ▪ Element {j}: {{'weights': {element["weights"]}, 'bias': {element['bias']}, 'activation': {element['activation']}, 'target_activation': {element['activation']}}}")
        print("\n")

print(f"\n------------------- Netzwerk (v.1) vor der Forward-Propagation mit initialisierten b, w -------------------\n")
print_array_structure(network1)


------------------- Netzwerk (v.1) vor der Forward-Propagation mit initialisierten b, w -------------------

🔹 Ebene 0:
  ▪ Element 0: {'weights': None, 'bias': None, 'activation': None, 'target_activation': None}
  ▪ Element 1: {'weights': None, 'bias': None, 'activation': None, 'target_activation': None}
  ▪ Element 2: {'weights': None, 'bias': None, 'activation': None, 'target_activation': None}
  ▪ Element 3: {'weights': None, 'bias': None, 'activation': None, 'target_activation': None}
  ▪ Element 4: {'weights': None, 'bias': None, 'activation': None, 'target_activation': None}
  ▪ Element 5: {'weights': None, 'bias': None, 'activation': None, 'target_activation': None}
  ▪ Element 6: {'weights': None, 'bias': None, 'activation': None, 'target_activation': None}
  ▪ Element 7: {'weights': None, 'bias': None, 'activation': None, 'target_activation': None}
  ▪ Element 8: {'weights': None, 'bias': None, 'activation': None, 'target_activation': None}


🔹 Ebene 1:
  ▪ Element 0: {'wei

## forward propagation
- funktioen definieren
  - aktivierungsfunktionen
- funktionen aufrufen

In [78]:
# aktivierungsfunktionen

def activation_relu(x): # hidden layers (werden hier die resultate nicht immer mit jedem layer höher?)
    return max(0, x)

def activation_sigmoid(x):
    return 1 / (1 + math.exp(-x))

def softmax(x, x_list): # output layer
    x_list = [math.exp(i) for i in x_list]
    return math.exp(x) / sum(x_list)

print(softmax(2.0, [2.0, 1.0, 0.1]))

0.6590011388859679


In [81]:
# forwardpropagation

def forward_propagation(pixel_bild, netzwerk): # im pronzip füllt diese fuktion das feld "activation" des dictinaries!
    print(f"Input Pixelbild: {pixel_bild}")

    new_network = netzwerk

    # INPUT-LAYER: den inputwert des inputlayers als activation setzen
    for neuron in range(network_dimension[0]):
        new_network[0][neuron]["activation"] = pixel_bild[neuron] 

    # HIDDEN-LAYERS und OUTPUT-LAYER: die informationen n-ten layers werden ans n+1-ten layer weitergegeben (für das letzte layer muss eine sigmoid-funktin verwendet werden damit man die klassifiezierung so durchführen kann dass  näher bei 1 oder näher bei 0 aufteilen kann)
    for n in range(1, len(new_network)):
        outputs = []

        # folgendes wird für jedes neuron eines layers gemacht
        for neuron in range(network_dimension[n]):
            prev_activations = [new_network[n-1][i]["activation"] for i in range(network_dimension[n-1])] # liste mit activations des vorherigen layers
            akt_weights = new_network[n][neuron]["weights"] # liste mit den weights eines neurons des 2ten layers
            akt_bias = new_network[n][neuron]["bias"]
            
            # output-layer
            if n == len(new_network)-1:
                outputs.append(sum([prev_activations[x] * akt_weights[x] for x in range(len(prev_activations))]) + akt_bias)
                if neuron == network_dimension[n]-1:
                    pixel_updated_output = [softmax(outputs[output], outputs) for output in range((network_dimension[n]))]
                    for sm in range(len(pixel_updated_output)):
                        new_network[n][sm]["activation"] = pixel_updated_output[sm] # summe aller activations aus dem letzen layer  
            
            # alle anderen hidden layers
            else:
                pixel_updated = activation_relu(sum([prev_activations[x] * akt_weights[x] for x in range(len(prev_activations))]) + akt_bias)
                new_network[n][neuron]["activation"] = pixel_updated # summe aller activations aus dem letzen layer  

    return new_network, [node["activation"] for node in new_network[-1]] # mit pixelbild ist hier der semantsche vektor des letzen layers gemeint, also der activations des letzen layers

print(f"\n------------------- Netzwerk (v.2) nach der Forward-Propagation mit initialisierten a -------------------\n")
print(print_array_structure(forward_propagation(training_data_sample[0], network1)[0]))
print(f"Outputs des letzten Layers: {(forward_propagation(training_data_sample[0], network1)[1])}")



------------------- Netzwerk (v.2) nach der Forward-Propagation mit initialisierten a -------------------

Input Pixelbild: [1, 0, 0, 1, 0, 0, 1, 0, 0]
🔹 Ebene 0:
  ▪ Element 0: {'weights': None, 'bias': None, 'activation': 1, 'target_activation': 1}
  ▪ Element 1: {'weights': None, 'bias': None, 'activation': 0, 'target_activation': 0}
  ▪ Element 2: {'weights': None, 'bias': None, 'activation': 0, 'target_activation': 0}
  ▪ Element 3: {'weights': None, 'bias': None, 'activation': 1, 'target_activation': 1}
  ▪ Element 4: {'weights': None, 'bias': None, 'activation': 0, 'target_activation': 0}
  ▪ Element 5: {'weights': None, 'bias': None, 'activation': 0, 'target_activation': 0}
  ▪ Element 6: {'weights': None, 'bias': None, 'activation': 1, 'target_activation': 1}
  ▪ Element 7: {'weights': None, 'bias': None, 'activation': 0, 'target_activation': 0}
  ▪ Element 8: {'weights': None, 'bias': None, 'activation': 0, 'target_activation': 0}


🔹 Ebene 1:
  ▪ Element 0: {'weights': [-0.

## prediction machen

In [ ]:
# prediction machen

def predict(image, network):
    return 0